# MLflow AI Gateway: Stay at the Frontier

## The Challenge

When new models (GPT-5, Claude Sonnet 4.5, Gemini 2.5) are released, platform teams need to **evaluate, compare, and gradually migrate** their apps — balancing **quality, latency, cost, and governance** — so they can keep their GenAI stack at the frontier without breaking production or rewriting code.

---

## What You'll Learn

**Part 1: Fundamentals** (5 min)
1. ✅ Automated tracing - every call logged
2. 📝 Prompt versioning - register and retrieve

**Part 2: Model Migration Workflow** (15 min)
3. 🔄 Baseline capture from current model
4. 🎯 Auto-optimize prompts for new model
5. 📊 Compare quality, latency, cost
6. 🚀 Gradual migration with A/B testing

**Key Benefit:** One inline API for everything - no code rewriting required!

## Setup

In [ ]:
import mlflowlite as mlflow
import os
import time
import random

# Set your API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Set experiment
mlflow.set_experiment("ai_gateway_demo")

print("✅ Setup complete")

# Part 1: Fundamentals

## Feature 1: Automated Tracing

Every LLM call is automatically traced to MLflow - no manual logging needed.

In [ ]:
# Simple LLM call - automatically traced!
@mlflow.trace
def classify_sentiment(text: str) -> str:
    """Classify sentiment using GPT-4"""
    response = mlflow.llm_call(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": f"Classify sentiment as positive/negative/neutral: {text}"}],
        temperature=0
    )
    return response['choices'][0]['message']['content'].lower().strip()

# Test it - this call is automatically logged to MLflow!
result = classify_sentiment("This product is amazing!")
print(f"Sentiment: {result}")
print("\n✅ Check MLflow UI - this trace was automatically logged!")
print("   Navigate to: http://localhost:5000 → Traces")

## Feature 2: Prompt Versioning

Version your prompts like code - register, retrieve, and track changes.

In [ ]:
# Register a prompt (Version 1)
prompt_v1 = mlflow.register_prompt(
    name="sentiment_classifier",
    template="""Classify the sentiment as positive, negative, or neutral.

Text: {{text}}

Answer:"""
)

print(f"✅ Registered prompt v1")
print(f"URI: {prompt_v1['uri']}")
print(f"\nTemplate:\n{prompt_v1['template']}")

In [ ]:
# Use the versioned prompt
@mlflow.trace
def classify_with_prompt(text: str) -> str:
    """Classify using versioned prompt"""
    # Format the prompt
    prompt_text = prompt_v1['template'].replace('{{text}}', text)
    
    response = mlflow.llm_call(
        model="openai/gpt-4",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0,
        prompt_name="sentiment_classifier"  # Links to prompt version
    )
    return response['choices'][0]['message']['content'].lower().strip()

# Test with v1
result = classify_with_prompt("The service was terrible.")
print(f"Result: {result}")

In [ ]:
# Register an improved prompt (Version 2)
prompt_v2 = mlflow.register_prompt(
    name="sentiment_classifier",
    template="""Classify the sentiment of the following text.

Text: {{text}}

Rules:
- Answer ONLY with: positive, negative, or neutral
- Use lowercase
- No explanation needed

Answer:"""
)

print(f"✅ Registered prompt v2")
print(f"URI: {prompt_v2['uri']}")
print(f"\n📊 You now have 2 versions tracked in MLflow!")

In [ ]:
# Retrieve specific version
retrieved_prompt = mlflow.get_prompt(name="sentiment_classifier", version=1)
print(f"Retrieved v1:\n{retrieved_prompt['template']}")

print("\n" + "="*50)

latest_prompt = mlflow.get_prompt(name="sentiment_classifier")  # Gets latest
print(f"\nLatest version:\n{latest_prompt['template']}")

print("\n✅ Prompt versioning works like Git for your prompts!")

# Part 2: Model Migration Workflow

## Scenario: GPT-4 → GPT-4o-mini Migration

You have a production app using GPT-4. A new model (GPT-4o-mini) is released that's **faster and cheaper**. You want to migrate without breaking production.

### Step 1: Your Production App

In [ ]:
# Register production prompt
production_prompt = mlflow.register_prompt(
    name="sentiment_production",
    template="""Classify sentiment: positive, negative, or neutral.

Text: {{text}}"""
)

# Production function (currently using GPT-4)
@mlflow.trace
def production_classifier(text: str, model: str = "openai/gpt-4") -> str:
    """Production sentiment classifier"""
    prompt_text = production_prompt['template'].replace('{{text}}', text)
    response = mlflow.llm_call(
        model=model,
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0,
        prompt_name="sentiment_production"
    )
    return response['choices'][0]['message']['content'].lower().strip()

# Test current production
result = production_classifier("Amazing product!")
print(f"GPT-4 Result: {result}")
print("\n✅ Current production: GPT-4")

### Step 2: Collect Baseline Data

Collect outputs from your current model on representative production data.

In [ ]:
# Representative production test cases
test_cases = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "The service was terrible and the food arrived cold.",
    "It was okay, nothing special but not bad either.",
    "I'm so disappointed with this purchase. Complete waste of money.",
    "Best experience ever! Highly recommend to everyone.",
    "The product works as described. No complaints.",
    "I can't believe how amazing this turned out to be!",
    "Worst customer support I've ever dealt with.",
    "It's fine for the price. Gets the job done.",
    "This exceeded all my expectations. Truly wonderful!"
]

# Collect baseline (GPT-4 outputs)
print("🔄 Collecting baseline outputs from GPT-4...\n")
baseline_outputs = []

for i, text in enumerate(test_cases, 1):
    result = production_classifier(text, model="openai/gpt-4")
    baseline_outputs.append(result)
    print(f"{i}. {text[:40]}... → {result}")

print(f"\n✅ Collected {len(baseline_outputs)} baseline outputs")

### Step 3: Test New Model (GPT-4o-mini)

Test the new model with the same prompt.

In [ ]:
# Test new model
print("🔄 Testing GPT-4o-mini with same prompt...\n")
new_model_outputs = []

for i, text in enumerate(test_cases, 1):
    result = production_classifier(text, model="openai/gpt-4o-mini")
    new_model_outputs.append(result)
    baseline_result = baseline_outputs[i-1]
    match = "✅" if result == baseline_result else "❌"
    print(f"{i}. {match} GPT-4o-mini: {result} (baseline: {baseline_result})")

# Calculate consistency
matches = sum(1 for new, base in zip(new_model_outputs, baseline_outputs) if new == base)
consistency = matches / len(baseline_outputs) * 100

print(f"\n📊 Consistency with baseline: {consistency:.0f}%")
if consistency < 90:
    print("⚠️  New model produces different outputs - needs optimization!")

### Step 4: Auto-Optimize Prompt for New Model

Use DSPy to automatically optimize the prompt for GPT-4o-mini.

In [ ]:
# Optimize prompt for new model
from mlflowlite.optimization import optimize_prompts

print("🎯 Optimizing prompt for GPT-4o-mini...\n")

# Prepare training data
training_data = [
    {"text": text, "expected_sentiment": baseline}
    for text, baseline in zip(test_cases[:5], baseline_outputs[:5])
]

# Run optimization
optimized_prompt_text = optimize_prompts(
    task="sentiment_classification",
    model="openai/gpt-4o-mini",
    training_data=training_data,
    original_prompt=production_prompt['template']
)

print("\n✅ Optimization complete!")
print(f"\n📝 Original:\n{production_prompt['template']}")
print(f"\n📝 Optimized:\n{optimized_prompt_text}")

In [ ]:
# Register optimized prompt
optimized_prompt = mlflow.register_prompt(
    name="sentiment_production",
    template=optimized_prompt_text
)

print(f"✅ Registered optimized prompt as new version")
print(f"URI: {optimized_prompt['uri']}")

### Step 5: Evaluate Optimized Version

In [ ]:
# Test with optimized prompt
@mlflow.trace
def optimized_classifier(text: str) -> str:
    """Classifier with optimized prompt for GPT-4o-mini"""
    prompt_text = optimized_prompt['template'].replace('{{text}}', text)
    response = mlflow.llm_call(
        model="openai/gpt-4o-mini",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0,
        prompt_name="sentiment_production"
    )
    return response['choices'][0]['message']['content'].lower().strip()

print("🔄 Testing GPT-4o-mini with optimized prompt...\n")
optimized_outputs = []

for i, text in enumerate(test_cases, 1):
    result = optimized_classifier(text)
    optimized_outputs.append(result)
    baseline_result = baseline_outputs[i-1]
    match = "✅" if result == baseline_result else "❌"
    print(f"{i}. {match} Optimized: {result} (baseline: {baseline_result})")

# Calculate improved consistency
matches = sum(1 for opt, base in zip(optimized_outputs, baseline_outputs) if opt == base)
improved_consistency = matches / len(baseline_outputs) * 100

print(f"\n📊 Before optimization: {consistency:.0f}%")
print(f"📊 After optimization: {improved_consistency:.0f}%")
print(f"\n🎉 Improvement: +{improved_consistency - consistency:.0f} percentage points!")

### Step 6: Compare Performance

In [ ]:
# Performance comparison
import pandas as pd

comparison = pd.DataFrame([
    {
        "Model": "GPT-4",
        "Prompt": "Original",
        "Consistency": "100%",
        "Latency": "~800ms",
        "Cost/1M tokens": "$30",
        "Status": "🟢 Production"
    },
    {
        "Model": "GPT-4o-mini",
        "Prompt": "Original",
        "Consistency": f"{consistency:.0f}%",
        "Latency": "~300ms",
        "Cost/1M tokens": "$0.60",
        "Status": "❌ Not Ready"
    },
    {
        "Model": "GPT-4o-mini",
        "Prompt": "Optimized",
        "Consistency": f"{improved_consistency:.0f}%",
        "Latency": "~300ms",
        "Cost/1M tokens": "$0.60",
        "Status": "✅ Ready"
    }
])

print("\n📊 Model Comparison:\n")
print(comparison.to_string(index=False))

print("\n💡 Key Insights:")
print("   • 2.5x faster")
print("   • 50x cheaper")
print("   • Same quality (after optimization)")
print("   • ✅ Ready for gradual migration!")

### Step 7: Gradual Migration with A/B Testing

Roll out the new model gradually using weighted routing.

In [ ]:
# Gradual migration function
@mlflow.trace
def production_classifier_v2(text: str, rollout_percentage: int = 20) -> dict:
    """
    Production classifier with gradual rollout
    
    Args:
        text: Input text
        rollout_percentage: % of traffic to new model (0-100)
    """
    # Weighted routing
    use_new_model = random.random() * 100 < rollout_percentage
    
    start_time = time.time()
    
    if use_new_model:
        # New model with optimized prompt
        result = optimized_classifier(text)
        model_used = "gpt-4o-mini (optimized)"
    else:
        # Current production model
        result = production_classifier(text, model="openai/gpt-4")
        model_used = "gpt-4 (baseline)"
    
    latency_ms = (time.time() - start_time) * 1000
    
    return {
        "sentiment": result,
        "model": model_used,
        "latency_ms": latency_ms
    }

print("✅ Gradual migration function ready!")

In [ ]:
# Simulate production traffic with 20% rollout
print("🚀 Simulating production traffic (20% rollout)...\n")

traffic_stats = {"gpt-4 (baseline)": 0, "gpt-4o-mini (optimized)": 0}

for i, text in enumerate(test_cases, 1):
    result = production_classifier_v2(text, rollout_percentage=20)
    traffic_stats[result["model"]] += 1
    
    print(f"{i}. Model: {result['model']:25s} | Sentiment: {result['sentiment']:8s} | Latency: {result['latency_ms']:.0f}ms")

print(f"\n📊 Traffic Distribution:")
for model, count in traffic_stats.items():
    percentage = count / len(test_cases) * 100
    print(f"   {model}: {count}/{len(test_cases)} ({percentage:.0f}%)")

print("\n💡 Migration Strategy:")
print("   1. Start: 5% → Monitor for 1 day")
print("   2. Then: 20% → Monitor for 2 days")
print("   3. Then: 50% → Monitor for 3 days")
print("   4. Finally: 100% → Full migration")
print("   5. Rollback if any issues detected")

### Step 8: Full Migration

After successful A/B testing, deploy to 100% of traffic.

In [ ]:
# Production v3: 100% new model
@mlflow.trace
def production_classifier_v3(text: str) -> str:
    """Production v3: Fully migrated to GPT-4o-mini"""
    return optimized_classifier(text)

# Test final production version
print("🎉 Production V3 - Fully migrated to GPT-4o-mini\n")

test_samples = [
    "Amazing product!",
    "Terrible experience.",
    "It's okay."
]

for text in test_samples:
    result = production_classifier_v3(text)
    print(f"'{text}' → {result}")

print("\n✅ Migration Complete!")
print("\n📈 Benefits Achieved:")
print("   • 2.5x faster responses")
print("   • 50x cost reduction")
print("   • Same quality maintained")
print("   • Zero production downtime")
print("   • No code rewriting")
print("   • All changes tracked in MLflow")

## 🎯 What You Achieved

### Part 1: Fundamentals ✅

1. **Automated Tracing**
   - Every LLM call automatically logged
   - Full observability with zero config
   - View all traces in MLflow UI

2. **Prompt Versioning**
   - Register prompts with versions
   - Retrieve any version on demand
   - Track all changes over time
   - Link traces to prompt versions

### Part 2: Production Model Migration ✅

You executed a **zero-downtime model migration**:

1. **Baseline Capture** - Collected outputs from GPT-4
2. **New Model Testing** - Evaluated GPT-4o-mini
3. **Automatic Optimization** - Rewrote prompts for new model
4. **Performance Comparison** - 2.5x faster, 50x cheaper
5. **Gradual Rollout** - A/B tested with 20% traffic
6. **Full Migration** - Deployed to 100%

---

### 🔑 Key Takeaways

| Feature | Benefit |
|---------|--------|
| **Inline API** | No mlflow.start_run() needed - everything is automatic |
| **Automated Tracing** | Zero-config observability for all LLM calls |
| **Prompt Versioning** | Git-like version control for prompts |
| **Auto-Optimization** | Prompts adapt to new models automatically |
| **Gradual Migration** | Risk-free rollouts with A/B testing |
| **Single API** | Switch models without rewriting code |

---

### 🚀 Next Steps

Use this workflow when:
- **GPT-5 is released** → Evaluate and migrate
- **Claude Sonnet 4.5 arrives** → Compare and optimize
- **Reducing costs** → Downgrade model, maintain quality
- **Better models available** → Upgrade seamlessly

**Your platform is ready to stay at the frontier! 🎯**

---

### 📚 Resources

- [MLflow Auto-rewrite Prompts](https://mlflow.org/docs/latest/genai/prompt-registry/rewrite-prompts/)
- [MLflow Evaluation Guide](https://mlflow.org/docs/latest/genai/eval-monitor/quickstart/)
- [Prompt Management](https://mlflow.org/docs/latest/genai/prompt-registry/create-edit-prompts/)